In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# 1. Tải dữ liệu MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# 2. Tiền xử lý
# Reshape dữ liệu để phù hợp với đầu vào của CNN
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32')
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1)).astype('float32')

# Chuẩn hóa dữ liệu giá trị pixel từ [0, 255] về [0, 1]
x_train /= 255
x_test /= 255

# Chuyển nhãn (label) sang dạng one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(f"Kích thước dữ liệu huấn luyện: {x_train.shape}")
print(f"Kích thước dữ liệu kiểm tra: {x_test.shape}")

In [ ]:
# 3. Xây dưng mô hình CNN
'''
Trong ví dụ này, chúng ta sẽ sử dụng một mạng CNN có kiến trúc đơn giản, bao gồm:
- Hai lớp tích chập, trong đó: lớp tích chập thứ nhất sử dụng 32 bộ lọc kích thước 3x3,
lớp tích chập thứ hai sử dụng 64 bộ lọc kích thước 3x3.
Cả hai lớp tích chập đều sử dụng stride = 2, padding = 0, và hàm kích hoạt ReLU.

Theo sau mỗi lớp tích chập là một lớp max pooling.

Hai lớp kết nối đầy đủ, trong đó: lớp kết nối đầy đủ thứ nhất có 128 neuron, và sử dụng hàm kích hoạt ReLU;
lớp kết nối đầy đủ thứ hai (lớp đầu ra) có 10 neuron, tương ứng với 10 lớp cần phân loại, và sử dụng hàm kích hoạt Softmax.
Đầu ra của CNN chính là xác suất để một hình ảnh thuộc về một trong 10 lớp từ 0 đến 9.
'''

In [ ]:
# Xây dựng mô hình CNN
model = Sequential([
    # Lớp tích chập thứ nhất với 32 bộ lọc kích thước 3x3, stride=2, padding =0
    Conv2D(32, (3, 3), strides=(2,2), activation='relu', input_shape=(28, 28, 1)),
    # Lớp max pooling với kích thước cửa sổ 2x2
    MaxPooling2D((2, 2)),

    #Lớp tích chập thứ 2 với 64 bộ lọc, kích thước 3x3, stride=2, padding=0
    Conv2D(64, (3, 3), strides=(2,2), activation='relu'),
    # Lớp max pooling với kích thước cửa sổ 2x2
    MaxPooling2D((2, 2)),

    #Làm phẳng dữ liệu để đưa vào lớp kết nối đầy đủ
    Flatten(),

    #Lớp kết nối đầy đủ thứ nhất với 128 neuron
    Dense(128, activation='relu'),

    #Lớp kết nối đầy đủ thứ hai với 10 neuron (số lớp cần phân loại)
    Dense(10, activation='softmax')

])

In [ ]:
# Tóm tắt kiến trúc
model.summary()

In [ ]:
# 4. Biên dịch mô hình
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 5. Huấn luyện mô hình
history = model.fit(
    x_train,
    y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

In [ ]:
# 6. Đánh giá mô hình
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Độ chính xác trên tập kiểm tra: {test_acc * 100:.2f}%')

In [ ]:
# 7. Lưu hình
model.save('my_model.h5')

In [ ]:
# 8. Vẽ biểu đồ accuracy và loss value trong quá trình huấn luyện
plt.figure(figsize=(12, 4))

# Biểu đồ độ chính xác
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Huấn luyện')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Độ chính xác mô hình')
plt.xlabel('Epoch')
plt.ylabel('Độ chính xác')
plt.legend()

# Biểu đồ mất mát
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Huấn luyện')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Mất mát mô hình')
plt.xlabel('Epoch')
plt.ylabel('Mất mát')
plt.legend()

plt.tight_layout()
plt.savefig('training_history.png')
plt.show()

In [ ]:
# 9. Hiển thị một số dự đoán
n_samples = 10
sample_indices = np.random.choice(len(x_test), n_samples, replace=False)

plt.figure(figsize=(15, 3))
for i, idx in enumerate(sample_indices):
    plt.subplot(1, n_samples, i + 1)
    # Convert từ (28, 28, 1) về (28, 28) để hiển thị
    plt.imshow(x_test[idx].reshape(28, 28), cmap='gray')

    # Dự đoán
    pred = model.predict(x_test[idx:idx+1])[0]
    pred_label = np.argmax(pred)
    true_label = np.argmax(y_test[idx])

    # Màu tiêu đề: xanh lá nếu đúng, đỏ nếu sai
    title_color = 'green' if pred_label == true_label else 'red'

    plt.title(f"T:{true_label} P:{pred_label}", color=title_color)
    plt.axis('off')

plt.tight_layout()
plt.savefig('predictions.png')
plt.show()